In [4]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy import sparse
from scipy import io as spio
from scipy.sparse import coo_array, csr_matrix, csc_matrix
from scipy import sparse
from cdlib import algorithms
from cdlib import evaluation
import sklearn
import math
import itertools
import random
import time
from pathlib import Path
import sys
sys.path.insert(0, '../')
from utils import *

In [5]:
n = 514386
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
"""    
alg_params = {
    "markov_clustering": {
        "inflation": [1.2, 1.5, 1.8, 2, 2.2, 2.5],
        "pruning_threshold": [0.01, 0.005, 0.001],
        "convergence_check_frequency": [100]
    },
}
"""


# For louvain only
alg_params = {
    "louvain": {
        "resolution": [1.0],
        "randomize": [314159, 
                      2718, 
                      98765, 
                      12345, 
                      20181218,
                      20181228,
                      20190103, 
                      20190107,
                      20190322,
                      20190831,
                      20200311,
                      20200329,
                      20210507,
                      20220104,
                      20230225,
                      20240402
                     ]
    }
}


"""
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}
"""

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.louvain(G,resolution=1.0,randomize=314159)', 0), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 1), ('algorithms.louvain(G,resolution=1.0,randomize=98765)', 2), ('algorithms.louvain(G,resolution=1.0,randomize=12345)', 3), ('algorithms.louvain(G,resolution=1.0,randomize=20181218)', 4), ('algorithms.louvain(G,resolution=1.0,randomize=20181228)', 5), ('algorithms.louvain(G,resolution=1.0,randomize=20190103)', 6), ('algorithms.louvain(G,resolution=1.0,randomize=20190107)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=20190322)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=20190831)', 9), ('algorithms.louvain(G,resolution=1.0,randomize=20200311)', 10), ('algorithms.louvain(G,resolution=1.0,randomize=20200329)', 11), ('algorithms.louvain(G,resolution=1.0,randomize=20210507)', 12), ('algorithms.louvain(G,resolution=1.0,randomize=20220104)', 13), ('algorithms.louvain(G,resolution=1.0,randomize=20230225)', 14), ('algorithms.louvain(G,resolution=1.0,randomiz

In [6]:
fileprefix = "/home/mth/Data/UNC_DATASET/Metis_Format/"
fname = "Samusik_allNetworkMetis"
graph_file = fileprefix + fname + ".mtx"
G = None
print(graph_file)
if Path(graph_file).is_file():
    #print("File found")
    with open(graph_file) as f:
        G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
        #G = nx.read_weighted_edgelist(f)
        coms = None
        count = 0
        #print(G.edges())
        for k in clustering_enumeration:
            try:
                t1 = time.time()
                coms = eval(k[0])
                t2 = time.time()
                print(k[0], len(coms.communities), "communities, ", t2-t1, "seconds")
                nelem = 0
                for l in coms.communities:
                    nelem = nelem + len(l)
                if nelem == len(G.nodes()):
                    write_clust_lst(coms.communities, fileprefix + fname + "." + str(count))
                    count = count + 1
                    #print("VALID PARTITION")
                    print("---")
                else:
                    print("INVALID PARTITION")
                    print("---")
            except Exception as e:
                print("UNSUCCESSFUL", k[0], e)

/home/mth/Data/UNC_DATASET/Metis_Format/Samusik_allNetworkMetis.mtx
algorithms.louvain(G,resolution=1.0,randomize=314159) 29 communities,  6379.906644582748 seconds
---
algorithms.louvain(G,resolution=1.0,randomize=2718) 29 communities,  6914.846333026886 seconds
---
algorithms.louvain(G,resolution=1.0,randomize=98765) 29 communities,  7349.276458263397 seconds
---
algorithms.louvain(G,resolution=1.0,randomize=12345) 29 communities,  4819.1437339782715 seconds
---
algorithms.louvain(G,resolution=1.0,randomize=20181218) 29 communities,  6935.060651063919 seconds
---
algorithms.louvain(G,resolution=1.0,randomize=20181228) 29 communities,  7558.088316202164 seconds
---
algorithms.louvain(G,resolution=1.0,randomize=20190103) 29 communities,  5768.725483417511 seconds
---
algorithms.louvain(G,resolution=1.0,randomize=20190107) 28 communities,  4824.365396261215 seconds
---
algorithms.louvain(G,resolution=1.0,randomize=20190322) 29 communities,  5677.399784088135 seconds
---
algorithms.louva

In [ ]:
fileprefix = "/home/mth/Data/UNC_DATASET/Metis_Format/"
fname = "Samusik_allNetworkMetis"
#graph_file = fileprefix + fname + ".edgelist"
graph_file = fileprefix + fname
G = None
print(graph_file)
if Path(graph_file).is_file():
    print("File found")
    with open(graph_file, "rU") as f:
        first_line = f.readline().strip('\n')
        tokens = first_line.split(" ")
        n = int(tokens[0])
        m = int(tokens[1])
        m = m - 1
        print(n, m)
        #nz_rows, nz_cols = np.nonzero(A)
        row = [-1]*m
        col = [-1]*m
        val = [0]*m
        for i in range(m):
            line = f.readline().strip('\n')
            tokens = line.split(" ")
            #print(tokens)
            row[i] = int(tokens[0])-1
            col[i] = int(tokens[1])-1
            val[i] = float(tokens[2])
        r = coo_array((val, (row, col)), shape=(n, n))
        #spio.mmwrite(graph_file+".mtx", r)